In [1]:
import gc , psutil
import time , argparse
from scripts.data_util.ModelData import (
     DataBlock ,block_process ,block_hist_norm,
     path_block_data,path_norm_data,block_mask,
     block_load_DB,save_dict_data)
from scripts.data_util.DataTank import DataTank
from scripts.function.basic import *
from scripts.util.environ import get_logger , DIR_data
from scripts.util.basic import timer
 
logger = get_logger()
DIR_block  = f'{DIR_data}/block_data'
DIR_hist_norm = f'{DIR_data}/hist_norm'

_save_start_dt , _save_end_dt = 20070101 , None
general_param = {
    'start_dt' : 20140701 , 'end_dt' : 20150630 , 'mask' : True ,
    #'start_dt' : None , 'end_dt' : None , 'mask' : True ,
} 
process_param = {
    'y' : {
        'DB_source'  : {'labels': {'inner_path' : ['10days/lag1' , '20days/lag1']}} , 
    },
    'trade_day' : {
        'DB_source'  : {'trade_day': {
            'inner_path':'day/trade' ,
            'feature' : ['adjfactor', 'close', 'high', 'low', 'open', 'vwap' , 'volume' , 'turn_fl']
        }} , 
    },
    # 'gp' : {}
}

parser = argparse.ArgumentParser(description='manual to this script')
parser.add_argument("--confirm", type=str, default='')
try:
    parser = parser.parse_args()
except:
    parser = parser.parse_args(args=[])
# if parser.confirm == 'no' or not input('You want to update data? print "yes" to confirm!').lower() == 'yes' : exit()


t1 = time.time()
logger.critical('Data Processing start!')
logger.error(f'{len(process_param)} datas :' + str(list(process_param.keys())))

key = 'y'
param = process_param[key]

usage: ipykernel_launcher.py [-h] [--confirm CONFIRM]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\jinmeng\AppData\Roaming\jupyter\runtime\kernel-v2-18188ud6CVBw2htdF.json
24-01-17 07:13:49|MOD:<ipython-input-1-cbbe7209d054>|: Data Processing start!
24-01-17 07:13:49|MOD:<ipython-input-1-cbbe7209d054>|: 2 datas :['y', 'trade_day']


In [3]:
process_param = {
    'y' : {
        'DB_source'  : {'labels': {'inner_path' : ['10days/lag1' , '20days/lag1']}} , 
    },
    'trade_day' : {
        'DB_source'  : {
            'trade_day': {            
                'db' : None ,  'inner_path':'day/trade' ,
                'feature' : ['adjfactor', 'close', 'high', 'low', 'open', 'vwap' , 'turn_fl'],
            }
        } , 
    },
    'trade_30m' : {
        'DB_source'  : {
            'trade_30m': {
                'db' : 'trade_Xmin' ,  'inner_path':'30min/trade' , 
                'feature' : ['minute' , 'close', 'high', 'low', 'open', 'volume', 'vwap'] ,
            },
            'trade_day': {
                'db' : None ,  'inner_path':'day/trade' ,
                'feature' : ['volume' , 'turn_fl' , 'preclose'] ,
            }
        } ,
    } , 
    'trade_15m' : {
        'DB_source'  : {
            'trade_15m': {
                'db' : 'trade_Xmin' ,  'inner_path':'30min/trade' , 
                'feature' : ['minute' , 'close', 'high', 'low', 'open', 'volume', 'vwap'] ,
            },
            'trade_day': {
                'db' : None ,  'inner_path':'day/trade' ,
                'feature' : ['volume' , 'turn_fl' , 'preclose'] ,
            }
        } ,
    } , 
    'trade_week' : {
        'DB_source'  : {
            'trade_day': {
                'db' : None ,  'inner_path':'day/trade' ,
                'feature' : ['adjfactor', 'preclose' ,'close', 'high', 'low', 'open', 'vwap' , 'turn_fl'],
            },
        } ,
    } , 
    # 'gp' : {}
}

key = 'y'
param = process_param[key]

BlockDict = block_load_DB(param['DB_source'] , **general_param)
# BlockDict[key].values.shape

labels blocks reading 10days/lag1 Data1D's ...... cost 0.41 secs
labels blocks reading 20days/lag1 Data1D's ...... cost 0.37 secs
labels blocks merging ...... cost 0.09 secs


In [14]:
BlockDict['labels'].secid

array([     1,      2,      4, ..., 603993, 603997, 603998])

In [3]:
def _type_abbr(key):
    if (key.startswith('trade_') and len(key)>6):
        return key[6:]
    elif key.startswith(('rtn_lag','res_lag')):
        return f'{key[:3]}{sum([int(s) for s in key[7:].split("_")])}'
    else:
        return key
    
def block_price_adjust(data_block , adjfactor = True , multiply = None , divide = None , 
                       price_feat = ['preclose' , 'close', 'high', 'low', 'open', 'vwap']):
    
    adjfactor = adjfactor and ('adjfactor' in data_block.feature)
    if multiply is None and divide is None and (not adjfactor): return data_block  

    if isinstance(price_feat , (str,)): price_feat = [price_feat]
    i_price = np.where(np.isin(data_block.feature , price_feat))[0]
    if len(i_price) == 0: return data_block
    v_price = data_block.values[...,i_price]

    if adjfactor : v_price = np.multiply(v_price , data_block.get(feature=['adjfactor']))
    if multiply  is not None: v_price = np.multiply(v_price , multiply)
    if divide    is not None: v_price = np.divide(v_price , divide)

    data_block.values[...,i_price] = v_price
    return data_block

def block_vol_adjust(data_block , multiply = None , divide = None , 
                     vol_feat = ['volume' , 'amount', 'turn_tt', 'turn_fl', 'turn_fr']):
    if multiply is None and divide is None: return data_block

    if isinstance(vol_feat , (str,)): vol_feat = [vol_feat]
    i_vol = np.where(np.isin(data_block.feature , vol_feat))[0]
    if len(i_vol) == 0: return data_block
    v_vol = data_block.values[...,i_vol]
    if multiply is not None: v_vol = np.multiply(v_vol , multiply)
    if divide   is not None: v_vol = np.divide(v_vol , divide)
    data_block.values[...,i_vol] = v_vol
    return data_block

def block_process(BlockDict , key):
    np.seterr(invalid='ignore' , divide = 'ignore')
    assert isinstance(BlockDict , dict) , type(BlockDict)
    assert all([isinstance(block , DataBlock) for block in BlockDict.values()]) , DataBlock
    
    key_abbr = _type_abbr(key)

    if key_abbr == 'y':
        final_feat = None
        data_block = BlockDict['labels']
        
    elif key_abbr == 'day':
        final_feat = ['open','close','high','low','vwap','turn_fl']
        data_block = block_price_adjust(BlockDict[key])

    elif key_abbr in ['15m','30m','60m']:
        final_feat = ['open','close','high','low','vwap','turn_fl']
        data_block = BlockDict[key]
        db_day = BlockDict['trade_day'].align_secid(data_block.secid)
        
        data_block = block_price_adjust(data_block,divide=db_day.get(feature='preclose'))
        data_block = block_vol_adjust(data_block,divide=db_day.get(feature='volume')/db_day.get(feature='turn_fl'),vol_feat='volume')
        
        data_block.rename_feature({'volume':'turn_fl'})
    elif key_abbr in ['week']:
        
        final_feat = ['open','close','high','low','vwap','turn_fl']
        num_days = 5
        data_block = BlockDict['trade_day']

        data_block=block_price_adjust(data_block)
        new_values = np.full(np.multiply(data_block.shape,(1,1,num_days,1)),np.nan)
        for i in range(num_days): new_values[:,num_days-1-i:,i] = data_block.values[:,:len(data_block.date)-num_days+1+i,0]
        data_block.update(values = new_values)
        data_block = block_price_adjust(data_block , adjfactor = False , divide=data_block.get(inday=0,feature='preclose'))
    else:
        raise Exception(key)
    
    data_block.align_feature(final_feat)
    np.seterr(invalid='warn' , divide = 'warn')
    return data_block

with timer(f'{key} blocks process') as t:
    ThisBlock = block_process(BlockDict , key)
with timer(f'{key} blocks masking') as t:   
    ThisBlock = block_mask(ThisBlock , **general_param)
with timer(f'{key} blocks saving ') as t:
    ThisBlock.save(path_block_data(key) , start_dt=_save_start_dt , end_dt=_save_end_dt)
with timer(f'{key} blocks norming') as t:
    #ThisBlock = DataBlock().read_npz(path_block_data(key))
    block_hist_norm(ThisBlock , key , save_path=path_norm_data(key) , **general_param)

trade_30m blocks process ...... cost 1.04 secs


In [ ]:
import copy
shape1 = (10,10,10) 
shape2 = (2,3,4)
a1 = np.random.rand(*shape1)
a2 = np.random.rand(*shape2)
a3 = copy.deepcopy(a1)
idx = np.ix_([0,4],[2,3,5],[2,9,3,7])
a1[idx] = a2
a3[idx] , a1[idx] , a2

In [2]:

t1 = time.time()
logger.critical('Data Processing start!')
logger.error(f'{len(process_param)} datas :' + str(list(process_param.keys())))

for key , param in process_param.items():
    tt1 = time.time()
    print(f'{time.ctime()} : {key} start ...')
    
    BlockDict = block_load_DB(param['DB_source'])

    with timer(f'{key} blocks process') as t:
        ThisBlock = block_process(BlockDict , process_key=key)

    with timer(f'{key} blocks process') as t:   
        ThisBlock = block_mask(ThisBlock , process_key=key)

    with timer(f'{key} blocks saving ') as t:
        ThisBlock.save(path_block_data(key) , start_dt=_save_start_dt , end_dt=_save_end_dt)
    
    with timer(f'{key} blocks norming') as t:
        #ThisBlock = DataBlock().read_npz(path_block_data(key))
        block_hist_norm(ThisBlock , key , path_norm_data(key) , **param)
    
    tt2 = time.time()
    print(f'{time.ctime()} : {key} finished! Cost {tt2-tt1:.2f} Seconds')

    del ThisBlock
    gc.collect()

t2 = time.time()
logger.critical('Data Processing Finished! Cost {:.2f} Seconds'.format(t2-t1))


24-01-10 12:16:14|MOD:<ipython-input-2-a7aa9b3fe77b>|: Data Processing start!
24-01-10 12:16:14|MOD:<ipython-input-2-a7aa9b3fe77b>|: 2 datas :['y', 'trade_day']


Wed Jan 10 12:16:14 2024 : y start ...
y blocks reading 10days/lag1 Data1D's ...... cost 10.99 secs
y blocks reading 20days/lag1 Data1D's ...... cost 12.28 secs
y blocks merging ...... cost 4.00 secs
y blocks process ...... cost 1.19 secs
y blocks saving  ...... cost 22.76 secs
y blocks norming ...... cost 0.00 secs
Wed Jan 10 12:17:05 2024 : y finished! Cost 51.23 Seconds
Wed Jan 10 12:17:05 2024 : trade_day start ...
trade_day blocks reading day/trade Data1D's ...... cost 37.06 secs
trade_day blocks merging ...... cost 0.28 secs
trade_day blocks process ...... cost 5.54 secs
trade_day blocks saving  ...... cost 25.52 secs
trade_day blocks norming ...... cost 38.61 secs
Wed Jan 10 12:18:52 2024 : trade_day finished! Cost 107.01 Seconds


24-01-10 12:18:52|MOD:<ipython-input-2-a7aa9b3fe77b>|: Data Processing Finished! Cost 158.63 Seconds
